<a href="https://colab.research.google.com/github/Ibrahim-Ayaz/Cassava-Image-End-to-end-Classification-Project-/blob/main/cassava_image_classification_project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Cassava Image Classification End-to-End Project with TensorFlow

Cassava consists of leaf images for the cassava plant depicting healthy and four (4) disease conditions; Cassava Mosaic Disease (CMD), Cassava Bacterial Blight (CBB), Cassava Greem Mite (CGM) and Cassava Brown Streak Disease (CBSD). Dataset consists of a total of 9430 labelled images. The 9430 labelled images are split into a training set (5656), a test set(1885) and a validation set (1889). The number of images per class are unbalanced with the two disease classes CMD and CBSD having 72% of the images.

For more, you can refer to the following link: https://arxiv.org/abs/1806.02987


## Check for GPU access

In [ ]:
# Confirm access to a GPU
!nvidia-smi

## Downloading `helper_functions.py` script

The script contains useful helper functions we need when calculating model metrics, as well as plotting model results such as ROC/loss curves and confusion matrices.

In [ ]:
# Download helper function script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

In [ ]:
# Import the necessary dependencies (functions) from the script
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys, make_confusion_matrix

## Downloading the Cassava dataset from TensorFlow Datasets

We're going to be downloading and loading the dataset from TensorFlow datasets: https://www.tensorflow.org/datasets/catalog/cassava

In [ ]:
# Check if our required dataset is in TensorFlow datasets
import tensorflow as tf
import tensorflow_datasets as tfds

datasets = tfds.list_builders()
print('cassava' in datasets)

In [ ]:
# Download/load the dataset
# Note: 'cassava' typically includes a train split and may include a validation/test split depending on TFDS version
import tensorflow_datasets as tfds

builder = tfds.builder('cassava')
available_splits = set(builder.info.splits.keys())
print('Available splits:', available_splits)

val_split = 'validation' if 'validation' in available_splits else ('test' if 'test' in available_splits else None)
if val_split is None:
    # Fallback: if no val/test split exists, we'll carve a small validation set out of train later.
    (train_data,), ds_info = tfds.load(
        name = 'cassava',
        split=['train'],
        as_supervised = True,
        with_info = True
    )
    test_data = None
else:
    (train_data, test_data), ds_info = tfds.load(
        name = 'cassava',
        split = ['train', val_split],
        as_supervised = True,
        with_info = True
    )


In [ ]:
# If TFDS didn't provide a validation/test split, create one from the train set
import tensorflow as tf

if test_data is None:
    # Shuffle once deterministically and split 90/10
    train_count = ds_info.splits['train'].num_examples
    val_count = int(0.1 * train_count)
    train_data = train_data.shuffle(buffer_size = min(10_000, train_count), seed = 42, reshuffle_each_iteration = False)
    test_data = train_data.take(val_count)
    train_data = train_data.skip(val_count)
    print(f'Created validation split from train: train = {train_count - val_count}, val = {val_count}')


## Visualising samples from the dataset

**Note:** When doing ML/DL experiments, it's important to familiarise yourself with the samples within the dataset; that way, you'll have an intuition of how your experiments should be laid out, and what types of prprocessing and model selection will be required. Most importantly, view **random** samples since randomness makes it powerful when visualing each feature/characteristic of an example.

The explorer data's motto: visualise, visualise, visualise!

In [ ]:
# Show samples from the dataset
tfds.show_examples(train_data, ds_info)

## Create efficient data preprocessing functions & pipelines for our modelling experiments

We've now viewed some random images of each label, let's start building some efficient data preprocessing functions and pipelines to make our experiments run computationally fast and efficient as much as possible: https://www.tensorflow.org/guide/data_performance

In [ ]:
from tensorflow.keras import layers

# Define data augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
], name = 'data_augmentation')

In [ ]:
# Create preprocessing function for our data
def preprocess_image(image, label, image_size = 224):
  image = tf.image.resize(image, [image_size, image_size]) # Reshape target image to 224x224
  image = tf.cast(image, dtype = tf.float32)
  image = data_augmentation(image) # Pass image through data augmentation layer
  return image, label

In [ ]:
# Compare between original and transformed image

# Get a sample and label from dataset
for image, label in train_data.take(1):
  transformed_image = preprocess_image(image = image, label = label)

# Get the transformed image and label from the tuple (the tuple comes from the preprocessing function above)
preprocessed_image, transformed_label = transformed_image

# Print out image before and after prepcrocessing & shapes
print(f'Original image before preprocessing: \n{image[:0]}')
print(f'Original image shape: \n{image.shape}')
print(f'Original image datatype: {image.dtype}')
print(f'Preprocessed image: \n{preprocessed_image[:0]}')
print(f'Preprocessed image shape: \n{preprocessed_image.shape}')
print(f'Preprocessed image datatype: \n{preprocessed_image.dtype}')

## Converting our datasets to prefected datasets and batched for faster computing

In [ ]:
# Map preprocessing function, shuffle and parallelise it to training data
train_data = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE)
train_data = train_data.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

# Validation/test pipeline
test_data = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

## Create a ModelCheckpoint callback to save our models' weights during training

In [ ]:
# Create model checkpoint callback path
checkpoint_path = 'cassava_model_checkpoint.weights.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path, save_weights_only = True, monitor = 'val_accuracy', save_best_only = True)

## Turning on mixed precision training for faster training time

When running an experiments with a large amount of data, it is important to turn on mixed precision training so that the training time is decreased whilst still maintaining the precision metric score: https://www.tensorflow.org/guide/mixed_precision

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16') # Set global data policy for mixed precision
mixed_precision.global_policy()

## Modelling Experiments

Now we're going to be conducting a series of modelling experiments to find out which model outperforms all other models, so that we can deploy it to a mobile app or to a web application for testing custom images.

And when doing experiments as such, it's always good to start with a baseline and use it as your benchmark to see if other models can beat it or not.

And always experiment as much as you can so you can discover the pattern in each modelling experiment, i.e. changed in model's metrics or observing what happens when you change hyperparameters; the experiment practitioner's motto: experiment, experiment, experiment!

### Model 0 (baseline): ResNetV2101

In [ ]:
# Setup data inputs
input_size = (224, 224, 3)

# Get number of classes
num_classes = ds_info.features['label'].num_classes

# Create feature extractor model with Functional API
base_model = tf.keras.applications.ResNet101V2(include_top = False, weights = 'imagenet', input_shape = input_size)
base_model.trainable = False

inputs = layers.Input(shape = (224, 224, 3), name = 'input_layer')
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation = 'relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation = 'softmax', name = 'output_layer')(x)
model_0 = tf.keras.Model(inputs, outputs)

# Conmpile model
model_0.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

# Get model summary
model_0.summary()

In [ ]:
# Fit
initial_epochs = 10
history_0 = model_0.fit(train_data, epochs = initial_epochs, steps_per_epoch = len(train_data), validation_data = test_data, validation_steps = int(0.15 * len(test_data)), callbacks = [create_tensorboard_callback(dir_name = './tensorflow_logs', experiment_name = 'cassava_all_modelling_experiments/model_0'), model_checkpoint])

In [ ]:
# Plot loss curves for baseline
plot_loss_curves(history = history_0)

In [ ]:
# Evaluate baseline on test data
model_0.evaluate(test_data)

In [ ]:
# Make predictions with baseline
model_0_preds = model_0.predict(test_data)
model_0_preds[:1]

In [ ]:
# Convert baseline preds to labels
labels = model_0_preds.argmax(axis = 1)
labels[:10]

In [ ]:
import numpy as np

# Get true labels from the test dataset
true_labels = []
for images, labels_batch in test_data.unbatch():
    true_labels.append(labels_batch.numpy())

true_labels = np.array(true_labels)

# Plot model 0's confusion matrix
make_confusion_matrix(y_true = true_labels, y_pred = labels)

### Fine-tuning baseline model

In [ ]:
# Unfreeze last 20 layers of feature-extractor model
base_model.trainable = True
for layer in base_model.layers[:-20]:
  layer.trainable = False

# Freeze all BatchNormalization layers within the last 20 layers
for layer in base_model.layers[-20:]:
  if isinstance(layer, layers.BatchNormalization):
    layer.trainable = False

# Compile
model_0.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5),
                metrics = ['accuracy'])

fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

# Fit fine-tuned model
history_0_fine_tune = model_0.fit(train_data, epochs = total_epochs, initial_epoch = initial_epochs, steps_per_epoch = len(train_data), validation_data = test_data, validation_steps = int(0.15 * len(test_data)), callbacks = [create_tensorboard_callback(dir_name = './tensorflow_logs', experiment_name = 'cassava_all_modelling_experiments/model_0_fine-tuned'), model_checkpoint])

In [ ]:
# Plot loss curves for fine-tuned baseline
plot_loss_curves(history = history_0_fine_tune)

In [ ]:
# Compare original and fine-tuned model historys for baseline
compare_historys(original_history = history_0, new_history = history_0_fine_tune, initial_epochs = initial_epochs)

### Model 1: EfficientNetV2B0

In [ ]:
# Setup data inputs
input_size = (224, 224, 3)

# Get number of classes
num_classes = ds_info.features['label'].num_classes

# Create feature extractor model with Functional API
base_model = tf.keras.applications.EfficientNetV2B0(include_top = False, weights = 'imagenet', input_shape = input_size)
base_model.trainable = False

inputs = layers.Input(shape = (224, 224, 3), name = 'input_layer')
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation = 'relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation = 'softmax', name = 'output_layer')(x)
model_1 = tf.keras.Model(inputs, outputs)

# Conmpile model
model_1.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

# Get model summary
model_1.summary()

In [ ]:
# Fit
initial_epochs = 10
history_1 = model_1.fit(train_data, epochs = initial_epochs, steps_per_epoch = len(train_data), validation_data = test_data, validation_steps = int(0.15 * len(test_data)), callbacks = [create_tensorboard_callback(dir_name = './tensorflow_logs', experiment_name = 'cassava_all_modelling_experiments/model_1'), model_checkpoint])

In [ ]:
# Plot loss curves for model 1
plot_loss_curves(history = history_1)

In [ ]:
# Evaluate model on test data
model_1.evaluate(test_data)

In [ ]:
# Make predictions with model 1
model_1_preds = model_1.predict(test_data)
model_1_preds[:1]

In [ ]:
# Convert model 1 preds to labels
labels = model_1_preds.argmax(axis = 1)
labels[:10]

In [ ]:
# Plot model 1's confusion matrix
make_confusion_matrix(y_true = true_labels, y_pred = labels)

### Fine-tuning model 1

In [ ]:
# Unfreeze last 20 layers of feature-extractor model
base_model.trainable = True
for layer in base_model.layers[:-20]:
  layer.trainable = False

# Freeze all BatchNormalization layers within the last 20 layers
for layer in base_model.layers[-20:]:
  if isinstance(layer, layers.BatchNormalization):
    layer.trainable = False

# Compile
model_1.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5),
                metrics = ['accuracy'])

fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

# Fit fine-tuned model
history_1_fine_tune = model_1.fit(train_data, epochs = total_epochs, initial_epoch = initial_epochs, steps_per_epoch = len(train_data), validation_data = test_data, validation_steps = int(0.15 * len(test_data)), callbacks = [create_tensorboard_callback(dir_name = './tensorflow_logs', experiment_name = 'cassava_all_modelling_experiments/model_1_fine-tuned'), model_checkpoint])

In [ ]:
# Plot fine-tuned model 1 loss curves
plot_loss_curves(history = history_1_fine_tune)

In [ ]:
# Compare original and fine-tuned model 1 historys
compare_historys(original_history = history_1, new_history = history_1_fine_tune, initial_epochs = initial_epochs)

### Model 2: EfficientNetV2B1

In [ ]:
# Setup data inputs
input_size = (224, 224, 3)

# Get number of classes
num_classes = ds_info.features['label'].num_classes

# Create feature extractor model with Functional API
base_model = tf.keras.applications.EfficientNetV2B1(include_top = False, weights = 'imagenet', input_shape = input_size)
base_model.trainable = False

inputs = layers.Input(shape = (224, 224, 3), name = 'input_layer')
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation = 'relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation = 'softmax', name = 'output_layer')(x)
model_2 = tf.keras.Model(inputs, outputs)

# Conmpile model
model_2.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

# Get model summary
model_2.summary()

In [ ]:
# Fit
initial_epochs = 10
history_2 = model_2.fit(train_data, epochs = initial_epochs, steps_per_epoch = len(train_data), validation_data = test_data, validation_steps = int(0.15 * len(test_data)), callbacks = [create_tensorboard_callback(dir_name = './tensorflow_logs', experiment_name = 'cassava_all_modelling_experiments/model_2'), model_checkpoint])

In [ ]:
# Plot loss curves for model 2
plot_loss_curves(history = history_2)

In [ ]:
# Evaluate model on test data
model_2.evaluate(test_data)

In [ ]:
# Make predictions with model 2
model_2_preds = model_2.predict(test_data)
model_2_preds[:1]

In [ ]:
# Convert model 2 preds to labels
labels = model_2_preds.argmax(axis = 1)
labels[:10]

In [ ]:
# Plot model 2's confusion matrix
make_confusion_matrix(y_true = true_labels, y_pred = labels)

### Fine-tuning model 2

In [ ]:
# Unfreeze last 20 layers of feature-extractor model
base_model.trainable = True
for layer in base_model.layers[:-20]:
  layer.trainable = False

# Freeze all BatchNormalization layers within the last 20 layers
for layer in base_model.layers[-20:]:
  if isinstance(layer, layers.BatchNormalization):
    layer.trainable = False

# Compile
model_2.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5),
                metrics = ['accuracy'])

fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

# Fit fine-tuned model
history_2_fine_tune = model_2.fit(train_data, epochs = total_epochs, initial_epoch = initial_epochs, steps_per_epoch = len(train_data), validation_data = test_data, validation_steps = int(0.15 * len(test_data)), callbacks = [create_tensorboard_callback(dir_name = './tensorflow_logs', experiment_name = 'cassava_all_modelling_experiments/model_2_fine-tuned'), model_checkpoint])

In [ ]:
# Plot fine-tuned model 1 loss curves
plot_loss_curves(history = history_2_fine_tune)

In [ ]:
# Compare orignal and fine-tuned historys for model 2
compare_historys(original_history = history_2, new_history = history_2_fine_tune, initial_epochs = initial_epochs)

### Model 3: ResNet152V2

In [ ]:
# Setup data inputs
input_size = (224, 224, 3)

# Get number of classes
num_classes = ds_info.features['label'].num_classes

# Create feature extractor model with Functional API
base_model = tf.keras.applications.ResNet152V2(include_top = False, weights = 'imagenet', input_shape = input_size)
base_model.trainable = False

inputs = layers.Input(shape = (224, 224, 3), name = 'input_layer')
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation = 'relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation = 'softmax', name = 'output_layer')(x)
model_3 = tf.keras.Model(inputs, outputs)

# Conmpile model
model_3.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

# Get model summary
model_3.summary()

In [ ]:
# Fit
initial_epochs = 10
history_3 = model_3.fit(train_data, epochs = initial_epochs, steps_per_epoch = len(train_data), validation_data = test_data, validation_steps = int(0.15 * len(test_data)), callbacks = [create_tensorboard_callback(dir_name = './tensorflow_logs', experiment_name = 'cassava_all_modelling_experiments/model_3'), model_checkpoint])

In [ ]:
# Plot loss curves for model 3
plot_loss_curves(history = history_3)

In [ ]:
# Evaluate model
model_3.evaluate(test_data)

In [ ]:
# Make predictions with model 3
model_3_preds = model_3.predict(test_data)
model_3_preds[:1]

In [ ]:
# Convert predictions to labels for model 3
labels = model_3_preds.argmax(axis = 1)
labels[:10]

In [ ]:
# Plot model 3's confusion matrix
make_confusion_matrix(y_true = true_labels, y_pred = labels)

### Fine-tuning model 3

In [ ]:
# Unfreeze last 20 layers of feature-extractor model
base_model.trainable = True
for layer in base_model.layers[:-20]:
  layer.trainable = False

# Freeze all BatchNormalization layers within the last 20 layers
for layer in base_model.layers[-20:]:
  if isinstance(layer, layers.BatchNormalization):
    layer.trainable = False

# Compile
model_3.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5),
                metrics = ['accuracy'])

fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

# Fit fine-tuned model
history_3_fine_tune = model_3.fit(train_data, epochs = total_epochs, initial_epoch = initial_epochs, steps_per_epoch = len(train_data), validation_data = test_data, validation_steps = int(0.15 * len(test_data)), callbacks = [create_tensorboard_callback(dir_name = './tensorflow_logs', experiment_name = 'cassava_all_modelling_experiments/model_3_fine-tuned'), model_checkpoint])

In [ ]:
# Plot fine-tuned model 3's loss curves
plot_loss_curves(history = history_3_fine_tune)

In [ ]:
# Compare orginal and fine-tuned historys for model 3
compare_historys(original_history = history_3, new_history = history_3_fine_tune, initial_epochs = initial_epochs)

### Model 4: Mobilenet_v3

In [ ]:
# Setup data inputs
input_size = (224, 224, 3)

# Get number of classes
num_classes = ds_info.features['label'].num_classes

# Create feature extractor model with Functional API
base_model = tf.keras.applications.MobileNetV3Large(include_top = False, weights = 'imagenet', input_shape = input_size)
base_model.trainable = False

inputs = layers.Input(shape = (224, 224, 3), name = 'input_layer')
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation = 'relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation = 'softmax', name = 'output_layer')(x)
model_4 = tf.keras.Model(inputs, outputs)

# Conmpile model
model_4.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

# Get model summary
model_4.summary()

In [ ]:
# Fit
initial_epochs = 10
history_4 = model_4.fit(train_data, epochs = initial_epochs, steps_per_epoch = len(train_data), validation_data = test_data, validation_steps = int(0.15 * len(test_data)), callbacks = [create_tensorboard_callback(dir_name = './tensorflow_logs', experiment_name = 'cassava_all_modelling_experiments/model_4'), model_checkpoint])

In [ ]:
# Plot loss curves
plot_loss_curves(history = history_4)

In [ ]:
# Evaluate model
model_4.evaluate(test_data)

In [ ]:
# Make predictions with model
model_4_preds = model_4.predict(test_data)
model_4_preds[:1]

In [ ]:
# Convert predictions to labels
labels = model_4_preds.argmax(axis = 1)
labels[:10]

In [ ]:
# Plot model 4's confusion matrix
make_confusion_matrix(y_true = true_labels, y_pred = labels)

### Fine-tuning model 4

In [ ]:
# Unfreeze last 20 layers of feature-extractor model
base_model.trainable = True
for layer in base_model.layers[:-20]:
  layer.trainable = False

# Freeze all BatchNormalization layers within the last 20 layers
for layer in base_model.layers[-20:]:
  if isinstance(layer, layers.BatchNormalization):
    layer.trainable = False

# Compile
model_4.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5),
                metrics = ['accuracy'])

fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

# Fit fine-tuned model
history_4_fine_tune = model_4.fit(train_data, epochs = total_epochs, initial_epoch = initial_epochs, steps_per_epoch = len(train_data), validation_data = test_data, validation_steps = int(0.15 * len(test_data)), callbacks = [create_tensorboard_callback(dir_name = './tensorflow_logs', experiment_name = 'cassava_all_modelling_experiments/model_4_fine-tuned'), model_checkpoint])

In [ ]:
# Plot fine-tuned model's loss curves
plot_loss_curves(history = history_4_fine_tune)

In [ ]:
# Compare original and fine-tuned model historys
compare_historys(original_history = history_4, new_history = history_4_fine_tune, initial_epochs = initial_epochs)

### Model 5: ConvNeXtBase

In [ ]:
# Setup data inputs
input_size = (224, 224, 3)

# Get number of classes
num_classes = ds_info.features['label'].num_classes

# Create feature extractor model with Functional API
base_model = tf.keras.applications.ConvNeXtBase(include_top = False, weights = 'imagenet', input_shape = input_size)
base_model.trainable = False

inputs = layers.Input(shape = (224, 224, 3), name = 'input_layer')
x = base_model(inputs, training = False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation = 'relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation = 'softmax', name = 'output_layer')(x)
model_5 = tf.keras.Model(inputs, outputs)

# Conmpile model
model_5.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

# Get model summary
model_5.summary()

In [ ]:
# Fit
initial_epochs = 10
history_5 = model_5.fit(train_data, epochs = initial_epochs, steps_per_epoch = len(train_data), validation_data = test_data, validation_steps = int(0.15 * len(test_data)), callbacks = [create_tensorboard_callback(dir_name = './tensorflow_logs', experiment_name = 'cassava_all_modelling_experiments/model_5'), model_checkpoint])

In [ ]:
# Plot loss curves
plot_loss_curves(history = history_5)

In [ ]:
# Evaluate model
model_5.evaluate(test_data)

In [ ]:
# Make predictions with model
model_5_preds = model_5.predict(test_data)
model_5_preds[:1]

In [ ]:
# Convert preds to labels
labels = model_5_preds.argmax(axis = 1)
labels[:10]

In [ ]:
# Plot model 5's confusion matrix
make_confusion_matrix(y_true = true_labels, y_pred = labels)

### Fine-tuning model 5

In [ ]:
# Unfreeze last 20 layers of feature-extractor model
base_model.trainable = True
for layer in base_model.layers[:-20]:
  layer.trainable = False

# Freeze all BatchNormalization layers within the last 20 layers
for layer in base_model.layers[-20:]:
  if isinstance(layer, layers.BatchNormalization):
    layer.trainable = False

# Compile
model_5.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-5),
                metrics = ['accuracy'])

fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

# Fit fine-tuned model
history_5_fine_tune = model_5.fit(train_data, epochs = total_epochs, initial_epoch = initial_epochs, steps_per_epoch = len(train_data), validation_data = test_data, validation_steps = int(0.15 * len(test_data)), callbacks = [create_tensorboard_callback(dir_name = './tensorflow_logs', experiment_name = 'cassava_all_modelling_experiments/model_5_fine-tuned'), model_checkpoint])

In [ ]:
# Plot loss curves for fine-tuned model
plot_loss_curves(history = history_5_fine_tune)

In [ ]:
# Compare orginal and fine-tuned model historys
compare_historys(original_history = history_5, new_history = history_5_fine_tune, initial_epochs = initial_epochs)

## Uploading all of our modelling experiments to TensorBoard

We're now going to be uploading all of our modelling experiments to TensorBoard to view each metric for all of our model.

In [ ]:
# Upload models' results to TensorBoard
!pip install -q tensorboard
%load_ext tensorboard
%tensorboard --logdir=/content/tensorflow_logs
%reload_ext tensorboard